In [1]:
import pandas as pd
import numpy as np

import os
import json
import datetime

from matplotlib import pyplot as plt
from scipy import stats, special

import utils
import links
import generators
import estimation

%load_ext autoreload
%autoreload 2

# Definitions of non-intuitive things

#### What is X?
X is a matrix of weather conditions generated in 1-InputGeneration.ipynb

The data are synthetic, but they're based on statistical properties of real weather data.

#### What is a scenario?
Different "scenarios" represent different failure processes used to generate the data. These could be constant, piecewise linear, etc.

Sceanrios are pre-defined in "inputs/scenarios.csv". Right now they're completely made up. Next steps are to use realistic assumptions about the lifetime and failure properties of grid assets (ultimately this piece goes in 2-ScenarioGeneration.ipynb - but for now that notebook is a mess and accomplishes nothing)



In [2]:
X = pd.read_csv('inputs/weather.csv', index_col='time')
X.head()

,Wind,Precip,Temp
time,,,
0,6.14,0.000,61.43
1,4.60,0.000,53.07
2,3.37,0.000,59.19
3,7.51,0.000,58.69
4,7.90,0.007,58.01


In [23]:
scenarios = pd.read_csv('inputs/scenarios.csv', index_col='name')
        
for i, scenario in scenarios.iterrows():
    params = generators.params(scenario, fleet_size=10000)
    failures, failure_rate = generators.failure_data(params, X, links.Link)
    
    if 'scenario%i'%(i) not in os.listdir('scenarios/'):
        os.mkdir(os.path.join('scenarios', 'scenario%i'%(i)))
        os.mkdir(os.path.join('scenarios', 'scenario%i'%(i), 'plots'))
        
    df = pd.DataFrame(index=failures.index)
    df['count'] = failures
    df['rate'] = failure_rate

    df.to_csv(os.path.join('scenarios','scenario%i'%(i),'failures.csv'), header=True)


# Scenario analysis
It takes quite a while to run through a scenario, let alone all of them. If you're just playing around, you can change "scenarios_list" to run only a subset of scenarios, or change "mcmc_stopping_criteria" to stop after fewer iterations. People will run ~1e5 or 1e6 iterations, but if the model is a good fit (big if, see 5.2) then it converges much faster.

Just a heads up. :-)

In [22]:
scenarios_list = scenarios.index
mcmc_stopping_criteria = lambda it: it<1e5

for i in scenarios_list:
    scenario = 'scenario%i'%(i)

    y = pd.read_csv(os.path.join('scenarios', scenario, 'failures.csv'), index_col='time')['count']

    x_vars = [[],['Wind',],['Wind','Temp',]]
    for var in x_vars:
        _X = X[var]

        model_name = 'constant'
        for v in var:
            model_name += '+'+v

        # -------------------------------
        # TUNING MCMC SAMPLER
        # Transition in the MCMC chain is a random walk where step size is gaussian and spherical
        # If the step size is too big, parameter updates are rarely accepted (poor mixing)
        # If the step size is too small, autocorrelation is high, convergence is slow
        # in both cases the chain needs to be longer.
        # Rule of thumb is to choose sigma so that acceptance rate is 25%
        out = estimation.get_existing_transition_params(scenario, model_name)
        if len(out) == 0:
            sigma, p0 = estimation.transition_function(_X, y, links.Link(), tol=0.1, chain_size=1000)
            estimation.save_chain_params(sigma, p0, scenario, model_name)
        else:
            sigma, p0 = out

        params, acceptance = estimation.metropolis(p0, _X, y, links.Link(), sig=sigma, stop=mcmc_stopping_criteria)

        params['acceptance'] = acceptance
        params.to_csv(os.path.join('scenarios', scenario, 'chains', '%s.csv'%(model_name)))

/Users/laureldunn/anaconda2/lib/python2.7/site-packages/scipy/stats/_multivariate.py:660: RuntimeWarning: covariance is not symmetric positive-semidefinite.
  out = random_state.multivariate_normal(mean, cov, size)


1000
1000
1000
{'constant': {'variables': ['constant'], 'sigma': [-0.13770600930498858, 0.23720349563046192], 'p0': [-5.96156557001104], 'time': '2019-08-21 21:44:18.775976'}}
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
1000
1000
1000
1000
1000
{'constant+Wind': {'variables': ['constant', 'slope.Wind', 'threshold.Wind'], 'sigma': [-0.09728587328578167, -1.0501689253418646e-06, 0.1842613980989062, 0.2634207240948814], 'p0': [-5.952449653608347, -0.0

KeyboardInterrupt: 